# Trabajo IA

## Selección de características para mejorar modelos predictivos

En este cuadernillo se realiza el tratamiento de datos del csv TITANIC, así como la implementación y comparación de varios algoritmos de búsqueda y entrenamiento.

### Tratamiento de datos

Se aplicaron los siguientes procesos de preprocesamiento de datos:

    •Normalización de variables predictoras: Se normalizaron las variables Age y Fare utilizando el escalador MinMaxScaler para asegurar que todas las características estén en la misma escala.

    •Codificación numérica de atributos discretos: Los atributos Sex, Embarked, Alone y Deck, que originalmente se presentaban como cadenas de texto, fueron codificados numéricamente utilizando las técnicas de OrdinalEncoder o LabelEncoder, según corresponda.

### Primer experimento para evaluar la capacidad predictiva del conjunto de variables completo

    Para evaluar la calidad de las soluciones que se obtienen con los algoritmos de búsqueda de variables, se realizó un primer experimento en el que se entrenó un modelo de clasificación utilizando todas las variables predictoras disponibles. Se utilizó un árbol de decisión de clasificación.

### Algoritmos de búsqueda implementados

    •Búsqueda secuencial hacia atrás (backward_sequential_search): Este algoritmo busca encontrar el mejor subconjunto de variables predictoras eliminando iterativamente la variable que más afecta el rendimiento del modelo.

    •Búsqueda secuencial hacia atrás mixta (backward_sequential_search_mixto): Similar al anterior, pero también considera añadir variables si se mejora el rendimiento del modelo.

### Algoritmos de entrenamiento

    •Árboles de decisión de clasificación (DecisionTreeClassifier): Un algoritmo de aprendizaje supervisado utilizado para clasificación.




## Tratamiento de datos

In [1]:
import numpy as np
np.random.seed(357823)

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler



model = RandomForestClassifier()

titanic = pd.read_csv('../data/titanic.csv')
titanic.head()

atributos_discretos = ['Initial', 'SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married','Pclass', 'Parch', 'Embarked', 
                       'Age_band', 'Family_Size', 'Alone']
atributos_continuos = ['Age', 'Fare']
atributos = titanic.loc[:, atributos_discretos + atributos_continuos]

objetivo = titanic['Survived']
objetivo.head()  # objetivo es una Series unidimensional


codificador_atributos_discretos = OrdinalEncoder()
codificador_atributos_discretos.fit(atributos[atributos_discretos])

print('Número de atributos detectados discretos:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados discretos:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

atributos[atributos_discretos] = codificador_atributos_discretos.transform(atributos[atributos_discretos])
atributos.head()


# El método fit_transform ajusta el codificador a los datos y, a continuación, codifica estos adecuadamente. 

normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)

# Como nos interesa conservar los atributos originales, realizamos la
# normalización sobre una copia del DataFrame de atributos
atributos_normalizados = atributos.copy()
atributos_normalizados[:] = normalizador.fit_transform(atributos_normalizados)
atributos_normalizados.head()

titanic = atributos_normalizados.copy()
titanic['Survived'] = objetivo
titanic.head()


Número de atributos detectados discretos: 13

Nombres de los atributos detectados discretos:
['Initial' 'SibSp' 'Deck' 'Fare_cat' 'Title' 'Sex' 'Is_Married' 'Pclass'
 'Parch' 'Embarked' 'Age_band' 'Family_Size' 'Alone']

Categorías detectadas de cada atributo:
Initial: [0 1 2 3 4]
SibSp: [0 1 2 3 4 5 8]
Deck: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T' 'U']
Fare_cat: [0 1 2 3]
Title: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
Sex: ['female' 'male']
Is_Married: [0 1]
Pclass: [1 2 3]
Parch: [0 1 2 3 4 5 6]
Embarked: ['C' 'Q' 'S']
Age_band: [0 1 2 3 4]
Family_Size: [ 0  1  2  3  4  5  6  7 10]
Alone: ['No' 'Yes']


,Initial,SibSp,Deck,Fare_cat,Title,Sex,Is_Married,Pclass,Parch,Embarked,Age_band,Family_Size,Alone,Age,Fare,Survived
0,0.00,0.166667,1.00,0.000000,0.6875,1.0,0.0,1.0,0.0,1.0,0.25,0.125,0.0,0.271174,0.014151,0
1,0.25,0.166667,0.25,1.000000,0.7500,0.0,1.0,0.0,0.0,0.0,0.50,0.125,0.0,0.472229,0.139136,1
2,0.50,0.000000,1.00,0.333333,0.5000,0.0,0.0,1.0,0.0,1.0,0.25,0.000,1.0,0.321438,0.015469,1
3,0.25,0.166667,0.25,1.000000,0.7500,0.0,1.0,0.0,0.0,1.0,0.50,0.125,0.0,0.434531,0.103644,1
4,0.00,0.000000,1.00,0.333333,0.6875,1.0,0.0,1.0,0.0,1.0,0.50,0.000,1.0,0.434531,0.015713,0


### Evaluación del modelo


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate


(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

clasificador_CART = DecisionTreeClassifier(random_state=42)
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'min_samples_split': range(5, 20, 5)
}
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)
print("Mejores parámetros:", búsqueda_en_rejilla.best_params_)
print("Mejor score:", búsqueda_en_rejilla.best_score_)


Mejores parámetros: {'max_depth': 4, 'min_samples_split': 5}
Mejor score: 0.8188637482242133


In [5]:
resultados_validación_cruzada = cross_validate(clasificador_CART,
                                               atributos_entrenamiento,
                                               objetivo_entrenamiento,
                                               scoring='balanced_accuracy',
                                               cv=10, 
                                               n_jobs=-1,)
#resultados_validación_cruzada
resultados_validación_cruzada['test_score'].mean()

0.7708467566025705

In [6]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

# Supongamos que atributos es un DataFrame con las características y objetivo es una Serie con la variable objetivo
# Ajustar el modelo
clasificador_CART = DecisionTreeClassifier(max_depth=4)
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Calcular la precisión del modelo
score = clasificador_CART.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_CART.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

# Imprimir los resultados
print(f'Variabes predictoras: \n{atributos_entrenamiento.columns}')
print()
print(f'Precisión: {score}')
print()
print(f'Valores: {values}')
print()
print(f'Matriz de confusión: \n{m_confusion}')


Variabes predictoras: 
Index(['Initial', 'SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married',
       'Pclass', 'Parch', 'Embarked', 'Age_band', 'Family_Size', 'Alone',
       'Age', 'Fare'],
      dtype='object')

Precisión: 0.8581460674157303

Valores: Survived
0    439
1    273
Name: count, dtype: int64

Matriz de confusión: 
[[386  53]
 [ 48 225]]


### KNN

In [12]:
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer([('normalizador',
                                   MinMaxScaler(feature_range=(0, 1)),
                                   atributos_discretos + atributos_continuos)])

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

tubería_kNN = Pipeline([('preprocesador', normalizador),
                        ('kNN', KNeighborsClassifier())])
rejilla_de_parámetros = {
    # Número de vecinos impar (tarea de clasificación binaria)
    'kNN__n_neighbors': range(1, 10, 2),
    # Considerar las distancias Manhattan y euclídea
    'kNN__metric': ['manhattan', 'euclidean']
}


búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

print("Mejores parámetros:", búsqueda_en_rejilla.best_params_)
print("Mejor score:", búsqueda_en_rejilla.best_score_)

Mejores parámetros: {'kNN__metric': 'manhattan', 'kNN__n_neighbors': 5}
Mejor score: 0.7248677248677249


In [16]:
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 5 ejemplos más cercanos
    n_neighbors=5,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)


# Calcular la precisión del modelo
score = clasificador_kNN.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_kNN.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

print(f'Variabes predictoras: {atributos_entrenamiento.columns}')
print(f'Precisión: {score}')
print(f'Valores: {values}')
print(f'Matriz de confusión: \n{m_confusion}')

Variabes predictoras: Index(['Initial', 'SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married',
       'Pclass', 'Parch', 'Embarked', 'Age_band', 'Family_Size', 'Alone',
       'Age', 'Fare'],
      dtype='object')
Precisión: 0.851123595505618
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[390  49]
 [ 57 216]]


## Algoritmo de búsqueda hacia atrás

In [18]:
#Esto parametros nos permite tener el minimo num de variables de predictoras y un mayor score
import funciones.BusquedaSecuencialAtras as bsa
bsatras = bsa.backward_sequential_search(titanic, 'Survived', model, 1, 2)
bsatras


,variables,size,score
2,"[SibSp, Deck, Title]",3,0.815710
3,"[SibSp, Deck, Title, Sex]",4,0.807111
4,"[SibSp, Deck, Fare_cat, Title, Sex]",5,0.804943
6,"[SibSp, Deck, Fare_cat, Title, Sex, Pclass, Ag...",7,0.801371
9,"[Initial, SibSp, Deck, Fare_cat, Title, Sex, P...",10,0.796483
10,"[Initial, SibSp, Deck, Fare_cat, Title, Sex, P...",11,0.794278
8,"[SibSp, Deck, Fare_cat, Title, Sex, Pclass, Em...",9,0.793193
11,"[Initial, SibSp, Deck, Fare_cat, Title, Sex, I...",12,0.792264
5,"[SibSp, Deck, Fare_cat, Title, Sex, Pclass]",6,0.791334
7,"[SibSp, Deck, Fare_cat, Title, Sex, Pclass, Ag...",8,0.789177


### Árbol de decisión de clasificación(DecisionTreeClassifier)



In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(bsatras)-1, 1):
    selected_variables = bsatras.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = titanic.loc[:, selected_variables]
    

    # Dividir los datos
    X = titanic[selected_variables]
    objetivo= titanic['Survived']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=4,  # Máxima profundidad del árbol
        min_samples_split=5  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=4)
    clasificador_CART.fit(X_train, y_train)

    # Calcular la precisión del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(y_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()

ValueError: Number of labels=891 does not match number of samples=712

### KNN

In [ ]:
from sklearn.naive_bayes import CategoricalNB
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 5 ejemplos más cercanos
    n_neighbors=5,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)
for k in range(0, len(bsatras)-1, 1):

    selected_variables = bsatras.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = titanic.loc[:, selected_variables]
    clasificador_kNN.fit(solucion, objetivo)
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)


    # Calcular la precisión del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')

Variabes predictoras: ['Initial', 'Pclass', 'Fare']
Precisión: 0.675645342312009
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[380 169]
 [120 222]]
Variabes predictoras: ['Initial', 'Is_Married', 'Pclass', 'Fare']
Precisión: 0.7014590347923682
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[523  26]
 [240 102]]
Variabes predictoras: ['Initial', 'Is_Married', 'Pclass', 'Embarked', 'Fare']
Precisión: 0.712682379349046
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[488  61]
 [195 147]]
Variabes predictoras: ['Initial', 'Is_Married', 'Pclass', 'Embarked', 'Alone', 'Fare']
Precisión: 0.7205387205387206
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[452  97]
 [152 190]]
Variabes predictoras: ['Initial', 'SibSp', 'Is_Married', 'Pclass', 'Embarked', 'Alone', 'Fare']
Precisión: 0.7205387205387206
Valores: Survived
0    549
1    342
Name: cou

## Algoritmo de búsqueda hacia atrás mixta

In [ ]:
import funciones.BusquedaSecuencialAtrasMixta as bsam
busq_atras_mixta = bsam.backward_sequential_mixed_search(titanic, 'Survived', model, 18, 10, 10)
busq_atras_mixta

,variables,size,score
11,"[Initial, Pclass, Fare, Fare_cat, Family_Size,...",9,0.824327
10,"[Initial, Pclass, Parch, Fare, Fare_cat, Famil...",9,0.824183
9,"[Initial, Pclass, Parch, Alone, Fare, Fare_cat...",9,0.824176
12,"[Initial, Fare, Fare_cat, Family_Size, Sex, Ti...",9,0.824106
13,"[Fare, Fare_cat, Family_Size, Sex, Title, Alon...",9,0.824056
8,"[Initial, Is_Married, Pclass, Parch, Alone, Fa...",10,0.823892
15,"[Fare_cat, Family_Size, Sex, Title, Alone, Par...",10,0.823534
14,"[Fare_cat, Family_Size, Sex, Title, Alone, Par...",9,0.823359
7,"[Initial, SibSp, Is_Married, Pclass, Parch, Al...",10,0.819757
16,"[Fare_cat, Family_Size, Sex, Title, Alone, Par...",11,0.819216


### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(busq_atras_mixta)-1, 1):
    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = titanic.loc[:, selected_variables]
    # Realizar la búsqueda secuencial hacia atrás
    model = DecisionTreeClassifier(max_depth=8)

    # Dividir los datos
    X = titanic[selected_variables]
    objetivo= titanic['Survived']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=4,  # Máxima profundidad del árbol
        min_samples_split=5  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=4)
    clasificador_CART.fit(X_train, y_train)

    # Calcular la precisión del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Imprimir los resultados
    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()  

Variabes predictoras: 
['Initial', 'Pclass', 'Fare', 'Fare_cat', 'Family_Size', 'Sex', 'Title', 'Alone', 'Parch']
Precisión: 0.8911335578002245
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[516  33]
 [ 64 278]]

Variabes predictoras: 
['Initial', 'Pclass', 'Parch', 'Fare', 'Fare_cat', 'Family_Size', 'Sex', 'Title', 'Alone']
Precisión: 0.8911335578002245
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[516  33]
 [ 64 278]]

Variabes predictoras: 
['Initial', 'Pclass', 'Parch', 'Alone', 'Fare', 'Fare_cat', 'Family_Size', 'Sex', 'Title']
Precisión: 0.8911335578002245
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[516  33]
 [ 64 278]]

Variabes predictoras: 
['Initial', 'Fare', 'Fare_cat', 'Family_Size', 'Sex', 'Title', 'Alone', 'Parch', 'Pclass']
Precisión: 0.8911335578002245
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[516  33]
 [ 64

### KNN

In [ ]:
from sklearn.naive_bayes import CategoricalNB
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 5 ejemplos más cercanos
    n_neighbors=5,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)
for k in range(0, len(busq_atras_mixta)-1, 1):

    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = titanic.loc[:, selected_variables]
    clasificador_kNN.fit(solucion, objetivo)
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)


    # Calcular la precisión del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')

Variabes predictoras: ['Fare_cat', 'Family_Size', 'Sex', 'Title', 'Alone', 'Parch', 'Pclass', 'Initial', 'Fare', 'Is_Married', 'SibSp', 'Deck', 'Embarked', 'Age', 'Age_band']
Precisión: 0.7699214365881033
Valores: Survived
0    549
1    342
Name: count, dtype: int64
Matriz de confusión: 
[[463  86]
 [119 223]]
